In [ ]:
from nltk.corpus import wordnet as wn
import matplotlib.pyplot as plt
import nltk
import math
import numpy as np
import pandas as pd
import string
import gensim,logging
import seaborn as sns
import imgkit


In [ ]:
df1=pd.read_csv('wordsim353/combined.csv',usecols=[0,1,2])
df2=pd.read_csv('wordsim353/set1.csv',usecols=[0,1,2])
df3=pd.read_csv('wordsim353/set2.csv',usecols=[0,1,2])
syn=pd.read_csv('synonyms.csv')
hyp=pd.read_csv('hypernyms.csv')
ant=pd.read_csv('antonmys.csv')
mer=pd.read_csv('meronyms.csv')

In [ ]:
def lc(x):
    sims=[0]
    for syn1 in wn.synsets(x[1]):
        for syn2 in wn.synsets(x[0]):
            if syn1.pos()=='n' and syn2.pos()=='n':
                #depth=max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())
                depth = 20
                dist=syn1.shortest_path_distance(syn2)
                sim=-math.log((dist + 1) / (2.0 * depth))
                sims.append(sim)
    return max(sims)
for df in [df1,df2,df3,syn,hyp,ant,mer]:
    df['lc']=df.apply(lc,axis=1)

In [ ]:
def wp(x):
    sims=[0]
    for syn1 in wn.synsets(x[0]):
        for syn2 in wn.synsets(x[1]):
            lcs= syn2.lowest_common_hypernyms(syn1,use_min_depth=True)
            if len(lcs)==0:
                sims.append(0)
            else:
                try:
                    lc=lcs[0]
                    n3=lc.max_depth()
                    n1,n2=syn1.shortest_path_distance(lc),syn2.shortest_path_distance(lc)
                    sim=2.*n3/(n1+n2+2.*n3)
                    sims.append(sim)
                except: pass
    return max(sims)
for df in [df1,df2,df3,syn,hyp,ant,mer]:
    df['wp']=df.apply(wp,axis=1)

In [ ]:
def liu(x):
    sims=[0]
    for syn1 in wn.synsets(x[0]):
        for syn2 in wn.synsets(x[1]):
            lcs= syn2.lowest_common_hypernyms(syn1,use_min_depth=True)
            if len(lcs)==0:
                sims.append(0)
            else:
                a,b=1,100
                lc=lcs[0]
                H=lc.max_depth()
                L=syn1.shortest_path_distance(syn2)
                sim=math.exp(-a*L)*(1-math.exp(-2*b*H))/(1+math.exp(-2*b*H))
                sims.append(sim)
    return max(sims)
for df in [df1,df2,df3,syn,hyp,ant,mer]:
    df['liu']=df.apply(liu,axis=1)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
wiki=pd.read_csv('wiki_data.csv')
corpus =wiki.text.replace('', np.nan).dropna()



In [ ]:
words=pd.concat([df1['Word 1'],df1['Word 2'],
                 df2['Word 1'],df2['Word 2'],
                 df3['Word 1'],df3['Word 2'],
                 syn['Word 1'],syn['Word 2'],
                 hyp['Word 1'],hyp['Word 2'],
                 ant['Word 1'],ant['Word 2'],
                 mer['Word 1'],mer['Word 2']]).unique()
glosses={}
for word in words:
    gloss=''
    for sn in wn.synsets(word):
        defi=sn.definition()
        gloss+= ' ' + defi
    glosses[word]=gloss

text=''
for key in glosses: text+= ' ' + glosses[key]
vocab=set(nltk.word_tokenize(text))
vectorizer = TfidfVectorizer(vocabulary=vocab)
X = vectorizer.fit_transform(corpus)

In [ ]:
vectors={}

for key in glosses:
    words=nltk.word_tokenize(glosses[key])
    idx=map(lambda x: vectorizer.vocabulary_[x],words)
    vectors[key]=np.array(X[:,idx].sum(1)).ravel()

In [ ]:
for df in [df1,df2,df3,syn,hyp,ant,mer]:
    df['lesk']=df.apply(lambda x: np.corrcoef(vectors[x[0]],vectors[x[1]])[1,0],axis=1)

In [ ]:
df1.corr(method='spearman').iloc[0,1:].plot()
df2.corr(method='spearman').iloc[0,1:].plot()
df3.corr(method='spearman').iloc[0,1:].plot()
plt.legend(['combined','set1','set2'])
plt.savefig('wn_corr.png')

In [ ]:
names=['syn','hyp','ant','mer']
for i,df in enumerate([syn,hyp,ant,mer]):
    html=df.style.background_gradient(cmap='RdBu',axis=0).render()
    imgkit.from_string(html, names[i]+'_wn.png')


In [ ]:
vocab=words=pd.concat([df1['Word 1'],df1['Word 2']]).unique()
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from nltk.corpus import reuters
sentences=reuters.sents()
i=0

i+=2
model = gensim.models.Word2Vec(sentences, min_count=5,iter=2,size=100,sg=1)
plt.figure(figsize=[10,10])
plt.imshow(model.wv.vectors[:10000,:].T,cmap='RdBu', vmin=-1, vmax=1,aspect=50)
plt.savefig('w2v_sg_i'+str(i))
#for df in [syn,hyp,ant,mer]:
#    df['w2v_'+str(i)]=df.apply(helper,axis=1)


for k in range(5):
    i+=2
    model.train(sentences,epochs=2,total_examples=model.corpus_count)
    plt.figure(figsize=[10,10])
    plt.imshow(model.wv.vectors[:10000,:].T,cmap='RdBu', vmin=-1, vmax=1,aspect=50)
    plt.savefig('w2v_sg_i'+str(i))
    #for df in [syn,hyp,ant,mer]:
    #    df['w2v_'+str(i)]=df.apply(lambda x:,axis=1)



In [ ]:
def helper(x):
    try:return model_wiki.similarity(x[0],x[1])
    except: return np.nan

wiki=pd.read_csv('wiki_data.csv')
wiki_sent=list(wiki.text.str.split(' '))
wiki_sent=filter(lambda x: type(x)==list,wiki_sent)


In [ ]:
i=2
model_wiki = gensim.models.Word2Vec(wiki_sent, min_count=2,iter=2,size=100,sg=1)
plt.figure(figsize=[10,10])
plt.imshow(model_wiki.wv.vectors[:10000,:].T,cmap='RdBu', vmin=-1, vmax=1,aspect=50)
#plt.savefig('w2v_i'+str(i))
for df in [syn,hyp,ant,mer]:
    df['w2v_sg_'+str(i)]=df.apply(helper,axis=1)


for k in range(2):
    i+=2
    model_wiki.train(wiki_sent,epochs=2,total_examples=model_wiki.corpus_count)
    plt.figure(figsize=[10,10])
    plt.imshow(model_wiki.wv.vectors[:10000,:].T,cmap='RdBu', vmin=-1, vmax=1,aspect=50)
    #plt.savefig('w2v_i'+str(i))
    for df in [syn,hyp,ant,mer]:
        df['w2v_sg_'+str(i)]=df.apply(helper,axis=1)

In [ ]:
def helper(x):
    try:return model.similarity(x[0].lower(),x[1].lower())
    except: return np.nan

In [ ]:
model = gensim.models.Word2Vec(wiki_sent, min_count=2,iter=5,size=100,sg=0)
for df in [df1,df2,df3,syn,hyp,ant,mer]:
    df['100_cbow']=df.apply(helper,axis=1)
print 1
model = gensim.models.Word2Vec(wiki_sent, min_count=2,iter=5,size=50,sg=0)
for df in [df1,df2,df3,syn,hyp,ant,mer]:
    df['50_cbow']=df.apply(helper,axis=1)
print 2
model = gensim.models.Word2Vec(wiki_sent, min_count=2,iter=5,size=100,sg=1)
for df in [df1,df2,df3,syn,hyp,ant,mer]:
    df['100_sg']=df.apply(helper,axis=1)
print 3
model = gensim.models.Word2Vec(wiki_sent, min_count=2,iter=5,size=50,sg=1)
for df in [df1,df2,df3,syn,hyp,ant,mer]:
    df['50_sg']=df.apply(helper,axis=1)
print 4
    

In [ ]:
df1.corr(method='spearman').iloc[0,1:].plot()
df2.corr(method='spearman').iloc[0,1:].plot()
df3.corr(method='spearman').iloc[0,1:].plot()
plt.legend(['combined','set1','set2'])
plt.savefig('w2v_corr.png')

In [ ]:
print df1.corr(method='spearman').iloc[0,1:]
print df2.corr(method='spearman').iloc[0,1:]
print df3.corr(method='spearman').iloc[0,1:]

In [ ]:
names=['syn','hyp','ant','mer']
for i,df in enumerate([syn,hyp,ant,mer]):
    html=df.style.background_gradient(cmap='RdBu').render()
    imgkit.from_string(html, names[i]+'_w2v.png')


In [ ]:
names=['syn','hyp','ant','mer']
for i,df in enumerate([syn,hyp,ant,mer]):
    html=df.style.background_gradient(cmap='RdBu').render()
    imgkit.from_string(html, names[i]+'_w2v_sg_epochs.png')
